In [1]:
import sys
# sys.path
import ee
import pprint
from IPython.display import Image
# from image import Image as local_model
import openet.ssebop as local_model

gridded C version


In [2]:
# ! --NotebookApp.iopub_data_rate_limit 2000000.0
# ! NotebookApp.iopub_data_rate_limitFloat 2000000.0
# !conda config --NotebookApp.iopub_data_rate_limit 2000000.0
# !jupyter notebook --NotebookApp.iopub_data_rate_limit 2000000.0

In [3]:
try:
    ee.Initialize()
    print('worked')
except ee.EEException:
    ee.Authenticate()
    ee.Initialize()

worked


In [4]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
image_size = 768

In [5]:
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_043035_20190715')
# matching ETo image
eto_img = ee.Image('projects/usgs-ssebop/pet/gridmet_median_v1/doy196')
landsat_crs = landsat_img.select('B3').projection().getInfo()['crs']
landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
landsat_dt = ee.Date(landsat_img.get('system:time_start'))

In [6]:
# Build the SSEBop object from the Landsat image
model_obj = local_model.Image.from_landsat_c1_toa(
    landsat_img,
    et_reference_source='IDAHO_EPSCOR/GRIDMET', 
    et_reference_band='etr', 
    et_reference_factor=0.85,
    et_reference_resample='nearest',
    tcorr_source='GRIDDED',
)

In [7]:
# ndvi_image = model_obj.ndvi
# print(ndvi_image.getInfo())

In [8]:
tcorr = model_obj.tcorr_gridded

In [9]:
# === USER ====
attempt = 'III'
level = '02'
name_root = f'rn{level}_vers{attempt}'

tcorr_name =  f'a_test_tcorr_{name_root}'
etf_name = f'a_test_etf_{name_root}'
eta_name = f'a_test_eta_{name_root}'

In [10]:
def ee_task_start(task, n=10):
    """Make an exponential backoff Earth Engine request"""
    output = None
    for i in range(1, n):
        try:
            task.start()
            break
        except Exception as e:
            time.sleep(i ** 2)
    return task

In [11]:
# try to export and see if the image looks like expected.
# task = ee.batch.Export.image.toDrive(image=tcorr_image, description='CFact_TestGTE', region=landsat_region)
task = ee.batch.Export.image.toAsset(image=tcorr , description=tcorr_name, assetId=f'users/gbrlparrish/griddedC/{tcorr_name}', region=landsat_region)
# start the tasktest
print('starting')
ee_task_start(task=task)
print('task submitted')
# task2 = ee.batch.Export.image.toAsset(image= tcorr_rn02_hot_testI, description='tcorr_rn02_hot_testI', assetId='users/gbrlparrish/griddedC/tcorr_rn02_hot_testI', region=landsat_region)
# # start the task
# print('starting task 2')
# ee_task_start(task=task2)
# print('task 2 submitted')
# # # 

starting
task submitted


In [12]:
ETf = model_obj.et_fraction
# make ETa by multiplying by gridmet ETo v1
ETa = ETf.multiply(eto_img)

In [13]:
# try to export EToF (Doesn't work. Error is "Error: Too many input pixels per output pixel. Need 27778, but only 64 allowed." Like Whaaaaat)
task = ee.batch.Export.image.toAsset(image=ETf, description=etf_name, assetId=f'users/gbrlparrish/griddedC/{etf_name}', region=landsat_region)
# start the task
print('starting')
ee_task_start(task=task)
print('task submitted')
# ============================
task = ee.batch.Export.image.toAsset(image=ETa, description=eta_name, assetId=f'users/gbrlparrish/griddedC/{eta_name}', region=landsat_region)
# start the task
print('starting')
ee_task_start(task=task)
print('task submitted')

# # TODO - export as asset and then display in the notebook.
# # # # =========================================================================
# # # # Try to show EToF and it also doesn't work
# # image_url = model_obj.et_fraction\
# #     .getThumbURL({'min': 0, 'max': 1.2, 'palette': et_palette, 
# #                   'region': landsat_region, 'dimensions': image_size})
# # Image(image_url, embed=True, format='png')

starting
task submitted
starting
task submitted
